In [22]:
import sys
sys.path.append('/Users/Ziqi/Desktop/developer/mgwr/')
from mgwr.gwr import GWR
from mgwr.FastGWR import FastGWR
from mgwr.sel_bw import Sel_BW
import pandas as pd
import numpy as np

In [23]:
zillow = pd.read_csv("/Users/Ziqi/Desktop/developer/gwr-hpc/zillow_15k.csv",sep=',')
zillow.head()
y = zillow.value.as_matrix().reshape(-1,1)
X = zillow.iloc[:,3:].as_matrix()
k = zillow.shape[1]
u = zillow.utmX
v = zillow.utmY
n = zillow.shape[0]
coords = list(zip(u,v))

In [ ]:
%%time 
fast_rslt = FastGWR(coords,y,X).fit(nproc=4) #FastGWR fitting with optimal bw seaching

mpiexec -n 4 python /Users/Ziqi/Desktop/developer/mgwr/mgwr/FastGWR_mpi.py -data /var/folders/h8/x0q8wyd52sz085tl9zwzrx7w0000gn/T/tmpobmjyutd -out /var/folders/h8/x0q8wyd52sz085tl9zwzrx7w0000gn/T/tmpdxkc4ynn
CPU times: user 455 ms, sys: 86.6 ms, total: 542 ms
Wall time: 3min 59s


In [ ]:
%%time 
bw = Sel_BW(coords,y,X).search(bw_min=45) #MGWR fitting with optimal bw seaching
rslt = GWR(coords,y,X,bw=bw).fit()

In [24]:
%%time
fast_rslt = FastGWR(coords,y,X,bw=122).fit(nproc=4) #FastGWR fitting with defined bandwidth

mpiexec -n 4 python /Users/Ziqi/Desktop/developer/mgwr/mgwr/FastGWR_mpi.py -data /var/folders/h8/x0q8wyd52sz085tl9zwzrx7w0000gn/T/tmpx73m5zu9 -out /var/folders/h8/x0q8wyd52sz085tl9zwzrx7w0000gn/T/tmppkjvul82 -bw 122
CPU times: user 540 ms, sys: 244 ms, total: 784 ms
Wall time: 17.6 s


In [25]:
%%time
rslt = GWR(coords,y,X,bw=122).fit() #MGWR fitting with defined bandwidth

CPU times: user 1min 28s, sys: 27 s, total: 1min 55s
Wall time: 2min 28s


In [20]:
#tr_S
print(rslt.tr_S)
print(fast_rslt.tr_S)

1041.6607662675278
1041.6607662675278


In [21]:
#AICc
print(rslt.aicc)
print(fast_rslt.aicc)

124374.05181029711
124374.0518102971


In [ ]:
#Sigma2
print(rslt.sigma2)
print(fast_rslt.sigma2)

In [ ]:
np.allclose(fast_rslt.params, rslt.params)

In [ ]:
np.allclose(fast_rslt.bse, rslt.bse)

In [ ]:
np.allclose(fast_rslt.predy, rslt.predy.reshape(-1))